In [2]:
import numpy as np   # basic numeric module in python, for array and matrix computation
import pandas as pd  # advanced numeric module, excels in data frame analysis
import matplotlib.pyplot as plt  # for data visualization
%pylab inline
import statsmodels.formula.api as smf    # for OLS regression

Populating the interactive namespace from numpy and matplotlib


## Example. Income vs Education in NYC zip code areas

Model average income per capita against percentages of individuals of different education level within the area

### Memo:
* **IncomePerCapita**----measured in USD
* **PopOver25** et al----population number under each category, e.g.
    * total population over 25 years old
    * holding a Bachelor's degree
    * graduating from professional school, etc.

In [3]:
data4 = pd.read_csv('data/IncomeEduReg.csv')
data4.head()

,Zipcode,IncomePerCapita,PopOver25,LessThanHS,HighSchool,SomeCollege,Bachelor,Master,Doctorate,ProfSchool
0,10001,77512.0,16328,1389,1665,2075,6061,3412,519,1207
1,10002,26905.0,60932,21170,12718,8532,12721,4001,641,1149
2,10003,79088.0,41182,1499,2810,4516,17958,9094,1626,3679
3,10004,98020.0,2279,29,87,305,984,550,86,238
4,10005,99633.0,5954,133,103,454,2745,1637,219,663


We will be looking to model the total income of the zip code as
$$
TotalIncome=\sum_i w_i Population_i
$$
where $Population_i$ is the number of people with certain education and the weight coefficient $w_i$ could be interpreted as the average income per capita within this population group. 

In [4]:
data4.dropna(inplace = True)  #drop NAN to avoid invalid computation

In [6]:
data4['Income']=data4['IncomePerCapita']*data4['PopOver25'] #calculate total income

In [21]:
#first run a regression again total topulation to define the overall average income per capita
lm2 = smf.ols(formula = 'Income ~ PopOver25-1', data = data4).fit()
print(lm2.summary())

                            OLS Regression Results                            
Dep. Variable:                 Income   R-squared:                       0.658
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     347.0
Date:                Wed, 02 Oct 2019   Prob (F-statistic):           7.54e-44
Time:                        16:32:21   Log-Likelihood:                -3976.7
No. Observations:                 181   AIC:                             7955.
Df Residuals:                     180   BIC:                             7959.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
PopOver25   3.217e+04   1726.829     18.627      0.0

And now run the regression for all the education categories

In [22]:
#use this operator to generate the regression formulae
' + '.join(data4.columns[3:10])

'LessThanHS + HighSchool + SomeCollege + Bachelor + Master + Doctorate + ProfSchool'

In [23]:
lm2 = smf.ols(formula = 'Income ~ ' + ' + '.join(data4.columns[3:10])+'-1', data = data4).fit()
print(lm2.summary())

                            OLS Regression Results                            
Dep. Variable:                 Income   R-squared:                       0.971
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                     840.7
Date:                Wed, 02 Oct 2019   Prob (F-statistic):          1.50e-130
Time:                        16:34:43   Log-Likelihood:                -3752.6
No. Observations:                 181   AIC:                             7519.
Df Residuals:                     174   BIC:                             7542.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
LessThanHS  -9811.5391   5433.915     -1.806      

Only two regressors seem significant according to p-value. This is likely because of the strong internal correlations between the regressors, especially among Bachelor,Master,Doctor and Prof.School

In [24]:
data4.corr()

,Zipcode,IncomePerCapita,PopOver25,LessThanHS,HighSchool,SomeCollege,Bachelor,Master,Doctorate,ProfSchool,Income
Zipcode,1.000000,-0.490675,0.089223,0.138386,0.298570,0.233799,-0.136994,-0.224106,-0.361479,-0.340607,-0.326244
IncomePerCapita,-0.490675,1.000000,-0.286507,-0.530034,-0.559255,-0.500068,0.262061,0.385013,0.447516,0.571962,0.542535
PopOver25,0.089223,-0.286507,1.000000,0.747408,0.833786,0.876240,0.691929,0.516295,0.350869,0.313306,0.470183
LessThanHS,0.138386,-0.530034,0.747408,1.000000,0.779932,0.693017,0.146826,-0.056804,-0.116706,-0.189604,-0.061991
HighSchool,0.298570,-0.559255,0.833786,0.779932,1.000000,0.918826,0.244020,0.028185,-0.135187,-0.187038,0.005785
SomeCollege,0.233799,-0.500068,0.876240,0.693017,0.918826,1.000000,0.389975,0.200074,0.014369,-0.042483,0.146218
Bachelor,-0.136994,0.262061,0.691929,0.146826,0.244020,0.389975,1.000000,0.919023,0.769090,0.810396,0.873868
Master,-0.224106,0.385013,0.516295,-0.056804,0.028185,0.200074,0.919023,1.000000,0.904758,0.913308,0.912174
Doctorate,-0.361479,0.447516,0.350869,-0.116706,-0.135187,0.014369,0.769090,0.904758,1.000000,0.889662,0.839830
ProfSchool,-0.340607,0.571962,0.313306,-0.189604,-0.187038,-0.042483,0.810396,0.913308,0.889662,1.000000,0.943770


In [15]:
#if we keep just the significant ones
lm2 = smf.ols(formula = 'Income ~ Bachelor + ProfSchool -1', data = data4).fit()
print(lm2.summary())

                            OLS Regression Results                            
Dep. Variable:                 Income   R-squared:                       0.966
Model:                            OLS   Adj. R-squared:                  0.965
Method:                 Least Squares   F-statistic:                     2509.
Date:                Wed, 02 Oct 2019   Prob (F-statistic):          1.16e-131
Time:                        13:06:14   Log-Likelihood:                -3769.0
No. Observations:                 181   AIC:                             7542.
Df Residuals:                     179   BIC:                             7548.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Bachelor    8.828e+04   5074.404     17.398      0.0

In [17]:
#but then rest of the population is unattended, so include it
lm2 = smf.ols(formula = 'Income ~ Bachelor + ProfSchool + PopOver25-1', data = data4).fit()
print(lm2.summary())

                            OLS Regression Results                            
Dep. Variable:                 Income   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                     1840.
Date:                Wed, 02 Oct 2019   Prob (F-statistic):          1.13e-133
Time:                        13:07:09   Log-Likelihood:                -3760.2
No. Observations:                 181   AIC:                             7526.
Df Residuals:                     178   BIC:                             7536.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Bachelor    4.232e+04   1.18e+04      3.582      0.0

In [31]:
#but if we run the regressions using some of the previously "insignificant" features alone, we can actually get it significant for them
lm2 = smf.ols(formula = 'Income ~ Master + PopOver25-1', data = data4).fit()
print(lm2.summary())

                            OLS Regression Results                            
Dep. Variable:                 Income   R-squared:                       0.928
Model:                            OLS   Adj. R-squared:                  0.927
Method:                 Least Squares   F-statistic:                     1146.
Date:                Wed, 02 Oct 2019   Prob (F-statistic):          9.62e-103
Time:                        16:42:25   Log-Likelihood:                -3836.3
No. Observations:                 181   AIC:                             7677.
Df Residuals:                     179   BIC:                             7683.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Master      3.198e+05   1.24e+04     25.782      0.0

In [34]:
#but if we run the regressions using some of the previously "insignificant" features alone, we can actually get it signifi
lm2 = smf.ols(formula = 'Income ~ Bachelor + Master + Doctorate + PopOver25-1', data = data4).fit()
print(lm2.summary())

                            OLS Regression Results                            
Dep. Variable:                 Income   R-squared:                       0.935
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     640.9
Date:                Wed, 02 Oct 2019   Prob (F-statistic):          4.15e-104
Time:                        16:43:06   Log-Likelihood:                -3825.9
No. Observations:                 181   AIC:                             7660.
Df Residuals:                     177   BIC:                             7673.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Bachelor    9.536e+04   2.11e+04      4.527      0.0

In [45]:
lm2 = smf.ols(formula = 'Income ~ LessThanHS + HighSchool + PopOver25-1', data = data4).fit()
print(lm2.summary())

                            OLS Regression Results                            
Dep. Variable:                 Income   R-squared:                       0.931
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                     799.9
Date:                Wed, 02 Oct 2019   Prob (F-statistic):          5.02e-103
Time:                        16:45:33   Log-Likelihood:                -3832.0
No. Observations:                 181   AIC:                             7670.
Df Residuals:                     178   BIC:                             7680.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
LessThanHS -9.766e+04   8080.556    -12.086      0.0

## In-class excercise

Q1. Perform regression vs Bachelor, "Advanced" which incorporated Master, Doctorate and ProfSchool and "Other" incorporating less than Bachelor

Q2. Visualize the regression fit by plotting the observation versus our prediction for the income per zip code